<a href="https://colab.research.google.com/github/AnuAgni/Natural-Language-Processing/blob/main/word2vec.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# word2vec - skip gram 
using tensor flow : https://www.tensorflow.org/tutorials/text/word2vec

##Setup

In [ ]:
import io
import re
import string
import tqdm
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers

In [ ]:
%load_ext tensorboard

In [ ]:
SEED = 42
AUTOTUNE = tf.data.AUTOTUNE

##Vectorizing a sentence

In [ ]:
#tokenize the sentence
sentence = "the wide road shimmered in the hot sun"
tokens =list(sentence.lower().split())
print(len(tokens))

8


In [ ]:
#Create a vocabulary to save mappings from tokens to integer indices
vocab,index = {} , 1           #indexing starts from one    #is it a vocab a list data type??? Dictionary data type - curly braces is for dictionary
vocab['<pad>'] = 0             #adding a padding token    //what is it used for?? to replace an index not mentioned? or the distance between consecutive index? 
for token in tokens:           #starting a loop : i=1 to i=8 (maybe to match natural numebrs loop has started with 1 instead of zero)
  if token not in vocab:       #checking whether word already in list or not, if not then
    vocab[token]=index         #adding the word with required index
    index+=1                   #i++
vocab_size = len(vocab)        #length of vocab i.e. no. of elements calculated and stored
print(vocab)                   #the elements the printed

{'<pad>': 0, 'the': 1, 'wide': 2, 'road': 3, 'shimmered': 4, 'in': 5, 'hot': 6, 'sun': 7}


In [ ]:
#Create an inverse vocabulary to save mappings from integer indices to tokens?
inverse_vocab={index: token for token, index in vocab.items()}      #loop starts from 0 to 8,element stored at that index in vocab is stored in inverse_vocab wrt index
print(inverse_vocab)

{0: '<pad>', 1: 'the', 2: 'wide', 3: 'road', 4: 'shimmered', 5: 'in', 6: 'hot', 7: 'sun'}


In [ ]:
#vectorizing sentence
example_sequence=[vocab[word] for word in tokens]  #how does this work?? why instead of element index is being printed?
print(example_sequence)

[1, 2, 3, 4, 5, 1, 6, 7]


## Generate skip-grams from one sentence

In [ ]:
window_size = 2                     #words to be taken before and after the center word
positive_skip_grams, _ = tf.keras.preprocessing.sequence.skipgrams(
      example_sequence,             #generated above
      vocabulary_size=vocab_size,   #8 in this case
      window_size=window_size,
      negative_samples=0)           #negative_samples is set to 0 here, as batching negative samples generated by this function requires a bit of code
print(len(positive_skip_grams))

26


In [ ]:
#Print a few positive skip-grams:
for target, context in positive_skip_grams[:5]: #?
  print(f"({target}, {context}): ({inverse_vocab[target]}, {inverse_vocab[context]})")

(3, 4): (road, shimmered)
(4, 5): (shimmered, in)
(4, 3): (shimmered, road)
(2, 1): (wide, the)
(1, 6): (the, hot)


## Negative sampling for one skip-gram

The skipgrams function returns all positive skip-gram pairs by sliding over a given window span. To produce additional skip-gram pairs that would serve as negative samples for training, you need to sample random words from the vocabulary. Use the tf.random.log_uniform_candidate_sampler function to sample num_ns number of negative samples for a given target word in a window. You can call the function on one skip-grams's target word and pass the context word as true class to exclude it from being sampled.

num_ns (the number of negative samples per a positive context word) in the [5, 20] range is shown to work best for smaller datasets, while num_ns in the [2, 5] range suffices for larger datasets.

In [ ]:
target_word, context_word = positive_skip_grams[0]                          #target and context word for one positive skip gram 
num_ns=4                                                                    #number if negative samples per context word
context_class=tf.reshape(tf.constant(context_word,dtype="int64"),(1,1))
negative_sampling_candidates,_,_=tf.random.log_uniform_candidate_sampler(
    true_classes=context_class,                                             #class that should be sampled as positive
    num_true=1,                                                             #each positive skip gram has 1 positive context class
    num_sampled=num_ns,                                                     #number of negative context word to sample
    unique=True,                                                            #all negative samples should be unique
    range_max=vocab_size,                                                   #pick index of samples from 0 to vocab size
    seed=SEED,                                                              #seed for reproducibility
    name="negative_sampling"                                                #name for this operation
)
print(negative_sampling_candidates)
print([inverse_vocab[index.numpy()] for index in negative_sampling_candidates])

tf.Tensor([2 1 4 3], shape=(4,), dtype=int64)
['wide', 'the', 'shimmered', 'road']


## Construct one training example

For a given positive (target_word, context_word) skip-gram, you now also have num_ns negative sampled context words that do not appear in the window size neighborhood of target_word. Batch the 1 positive context_word and num_ns negative context words into one tensor. This produces a set of positive skip-grams (labeled as 1) and negative samples (labeled as 0) for each target word.

In [ ]:
# Add a dimension so you can use concatenation (in the next step).
negative_sampling_candidates = tf.expand_dims(negative_sampling_candidates, 1)

# Concatenate a positive context word with negative sampled words.
context = tf.concat([context_class, negative_sampling_candidates], 0)

# Label the first context word as `1` (positive) followed by `num_ns` `0`s (negative).
label = tf.constant([1] + [0]*num_ns, dtype="int64")

# Reshape the target to shape `(1,)` and context and label to `(num_ns+1,)`.
target = tf.squeeze(target_word)
context = tf.squeeze(context)
label = tf.squeeze(label)

In [ ]:
print(f"target_index    : {target}")
print(f"target_word     : {inverse_vocab[target_word]}")
print(f"context_indices : {context}")
print(f"context_words   : {[inverse_vocab[c.numpy()] for c in context]}")
print(f"label           : {label}")

target_index    : 3
target_word     : road
context_indices : [4 2 1 4 3]
context_words   : ['shimmered', 'wide', 'the', 'shimmered', 'road']
label           : [1 0 0 0 0]


A tuple of (target, context, label) tensors constitutes one training example for training your skip-gram negative sampling word2vec model. Notice that the target is of shape (1,) while the context and label are of shape (1+num_ns,)

In [ ]:
print("target  :", target)
print("context :", context)
print("label   :", label)

target  : tf.Tensor(3, shape=(), dtype=int32)
context : tf.Tensor([4 2 1 4 3], shape=(5,), dtype=int64)
label   : tf.Tensor([1 0 0 0 0], shape=(5,), dtype=int64)


##Skip gram sampling table

In [ ]:
sampling_table = tf.keras.preprocessing.sequence.make_sampling_table(size=10)
print(sampling_table)

[0.00315225 0.00315225 0.00547597 0.00741556 0.00912817 0.01068435
 0.01212381 0.01347162 0.01474487 0.0159558 ]


## Generate training data

In [ ]:
# Generates skip-gram pairs with negative sampling for a list of sequences
# (int-encoded sentences) based on window size, number of negative samples
# and vocabulary size.
def generate_training_data(sequences, window_size, num_ns, vocab_size, seed):
  # Elements of each training example are appended to these lists.
  targets, contexts, labels = [], [], []

  # Build the sampling table for `vocab_size` tokens.
  sampling_table = tf.keras.preprocessing.sequence.make_sampling_table(vocab_size)

  # Iterate over all sequences (sentences) in the dataset.
  for sequence in tqdm.tqdm(sequences):

    # Generate positive skip-gram pairs for a sequence (sentence).
    positive_skip_grams, _ = tf.keras.preprocessing.sequence.skipgrams(
          sequence,
          vocabulary_size=vocab_size,
          sampling_table=sampling_table,
          window_size=window_size,
          negative_samples=0)

    # Iterate over each positive skip-gram pair to produce training examples
    # with a positive context word and negative samples.
    for target_word, context_word in positive_skip_grams:
      context_class = tf.expand_dims(
          tf.constant([context_word], dtype="int64"), 1)
      negative_sampling_candidates, _, _ = tf.random.log_uniform_candidate_sampler(
          true_classes=context_class,
          num_true=1,
          num_sampled=num_ns,
          unique=True,
          range_max=vocab_size,
          seed=SEED,
          name="negative_sampling")

      # Build context and label vectors (for one target word)
      negative_sampling_candidates = tf.expand_dims(
          negative_sampling_candidates, 1)

      context = tf.concat([context_class, negative_sampling_candidates], 0)
      label = tf.constant([1] + [0]*num_ns, dtype="int64")

      # Append each element from the training example to global lists.
      targets.append(target_word)
      contexts.append(context)
      labels.append(label)

  return targets, contexts, labels